# GPT-1 모델


### 1. Encoder block 제거 
   - decoder만을 사용해 이전 토큰들을 기반으로 순차적으로 다음 토큰을 예측
   - gpt-1은 deocder 부분만 필요로 함으로 기존 transformer 그림에서 encoder 부분을 제거

### 2. positional encoding layer
   - 기존 transformer에서는 positional embedding을 cos,sin을 사용하여 구했지만 
   - gpt-1에서는 cos,sin으로 구하지 않고 위치 인코딩을 네트워크가 학습할 수 있도록 임베딩 레이어의 일부로 포함시킴

### 3. cross attention 제거 
   - gpt-1 model은 encoder 부분이 필요 없어짐으로 decoder가 encoder로 부터 받는 key,value값을 전달하는 cross attention 제거
   


In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Layer, Dense, LayerNormalization, Dropout
from tensorflow.keras import activations


## 1. 데이터 수집하기

In [2]:
data = pd.read_csv('../../Exploration/Ex07/data/data.csv', encoding ='cp949')

In [3]:
data.head()

Q            A  label
0           12시 땡!   하루가 또 가네요.      0
1      1지망 학교 떨어졌어    위로해 드립니다.      0
2     3박4일 놀러가고 싶다  여행은 언제나 좋죠.      0
3  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.      0
4          PPL 심하네   눈살이 찌푸려지죠.      0

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       11823 non-null  object
 1   A       11823 non-null  object
 2   label   11823 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.2+ KB


In [5]:
num_samples = data.shape[0]  # 또는 len(transformer_data)
print(f'Number of samples: {num_samples}')

Number of samples: 11823


## 2. 데이터 전처리 하기
- 지난 transformer 프로젝트 수행시 전처리 관련해서 유의미한 내용만 작성.

In [6]:
MAX_SAMPLES = 11823

In [7]:
# 전처리 함수
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)   # 각종 구두점과의 거리 만들기
    sentence = re.sub(r'[" "]+', " ", sentence)  # 공백이 두칸이면 한칸으로..
    sentence = re.sub(r'[^a-zA-Z0-9.,?!가-힣]', ' ', sentence) # 한글과 영어, 구두점을 제외한 모든 문자 공백으로 대체
    
    sentence = sentence.strip() 
    return sentence

In [8]:
questions = data['Q'].apply(preprocess_sentence).tolist() # 전처리 결과를 각각 리스트로 변환하여 questions와 answers에 저장
answers = data['A'].apply(preprocess_sentence).tolist()

In [9]:
print('전처리 후의 22번째 질문 샘플: {}'.format(questions[21]))
print('전처리 후의 22번째 답변 샘플: {}'.format(answers[21]))

전처리 후의 22번째 질문 샘플: 가스비 장난 아님
전처리 후의 22번째 답변 샘플: 다음 달에는 더 절약해봐요 .


## 3. SubwordTextencoder 사용하기

In [10]:
# 질문과 답변 데이터셋에 대해서 Vocabulary 생성
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)

In [11]:
# 시작 토큰과 종료 토큰에 고유한 정수를 부여합니다.
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

In [12]:
print('START_TOKEN의 번호 :' ,[tokenizer.vocab_size])
print('END_TOKEN의 번호 :' ,[tokenizer.vocab_size + 1])

START_TOKEN의 번호 : [8164]
END_TOKEN의 번호 : [8165]


In [13]:
VOCAB_SIZE = tokenizer.vocab_size + 2

In [14]:
print('정수 인코딩 후의 21번째 질문 샘플: {}'.format(tokenizer.encode(questions[21])))
print('정수 인코딩 후의 21번째 답변 샘플: {}'.format(tokenizer.encode(answers[21])))

정수 인코딩 후의 21번째 질문 샘플: [5758, 610, 2487, 4159]
정수 인코딩 후의 21번째 답변 샘플: [2355, 7504, 7, 6268, 97, 1]


In [15]:
# q,a를 합침으로 10 + 10 + $, 공백 까지 고려해서 22로 설정

MAX_LENGTH = 22
print(MAX_LENGTH)

22


In [16]:
# gpt 모델에서 사용 x
# transformer 정수 인코딩, 최대 길이를 초과하는 샘플 제거, 패딩
"""
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []
  
  for (sentence1, sentence2) in zip(inputs, outputs):
    # 정수 인코딩 과정에서 시작 토큰과 종료 토큰을 추가
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    # 최대 길이 40 이하인 경우에만 데이터셋으로 허용
    if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
      tokenized_inputs.append(sentence1)
      tokenized_outputs.append(sentence2)
  
  # 최대 길이 40으로 모든 데이터셋을 패딩
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
  
  return tokenized_inputs, tokenized_outputs

"""

"\ndef tokenize_and_filter(inputs, outputs):\n  tokenized_inputs, tokenized_outputs = [], []\n  \n  for (sentence1, sentence2) in zip(inputs, outputs):\n    # 정수 인코딩 과정에서 시작 토큰과 종료 토큰을 추가\n    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN\n    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN\n\n    # 최대 길이 40 이하인 경우에만 데이터셋으로 허용\n    if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:\n      tokenized_inputs.append(sentence1)\n      tokenized_outputs.append(sentence2)\n  \n  # 최대 길이 40으로 모든 데이터셋을 패딩\n  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(\n      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')\n  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(\n      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')\n  \n  return tokenized_inputs, tokenized_outputs\n\n"

In [17]:
# gpt 토크나이징 및 필터링
def tokenize_and_filter(dialogues):
    tokenized_inputs = []
  
    for dialogue in dialogues:
        question = dialogue['question']
        answer = dialogue['answer']
        
        # GPT는 질문과 답변을 하나의 시퀀스로 처리. 구분자로 '$'를 사용
        dialogue_sequence = question + " $" + answer
        
        # 정수 인코딩 과정
        tokenized_input = START_TOKEN + tokenizer.encode(dialogue_sequence) + END_TOKEN
        
        # 최대 길이 이하인 시퀀스만 허용
        if len(tokenized_input) <= MAX_LENGTH:
            tokenized_inputs.append(tokenized_input)
    
    # 패딩 처리 (최대 길이 기준)
    tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_inputs, maxlen=MAX_LENGTH, padding='post'
    )
  
    return tokenized_inputs

In [18]:
# 질문과 답변을 하나의 시퀀스로 결합하여 처리
dialogues = [{'question': q, 'answer': a} for q, a in zip(questions, answers)]
tokenized_inputs = tokenize_and_filter(dialogues)

print(f'단어장의 크기: {VOCAB_SIZE}')
print(f'필터링 후의 데이터 샘플 개수: {len(tokenized_inputs)}')


단어장의 크기: 8166
필터링 후의 데이터 샘플 개수: 11324


## Positional Encoding, Attention

In [19]:
# 포지셔널 인코딩 레이어 , transformerdptjsms Positional Encoding을 sin cos으로 구하였지만 ,gpt-1에서는 layer를 통해 position을 구함
"""
class PositionalEncoding(tf.keras.layers.Layer):
 
  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

     
  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles



  def positional_encoding(self, position, d_model):
    # 각도 배열 생성
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)

    # 배열의 짝수 인덱스에는 sin 함수 적용
    sines = tf.math.sin(angle_rads[:, 0::2])
    # 배열의 홀수 인덱스에는 cosine 함수 적용
    cosines = tf.math.cos(angle_rads[:, 1::2])

    # sin과 cosine이 교차되도록 재배열
    pos_encoding = tf.stack([sines, cosines], axis=0)
    pos_encoding = tf.transpose(pos_encoding,[1, 2, 0]) 
    pos_encoding = tf.reshape(pos_encoding, [position, d_model])

    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]
"""

'\nclass PositionalEncoding(tf.keras.layers.Layer):\n \n  def __init__(self, position, d_model):\n    super(PositionalEncoding, self).__init__()\n    self.pos_encoding = self.positional_encoding(position, d_model)\n\n     \n  def get_angles(self, position, i, d_model):\n    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))\n    return position * angles\n\n\n\n  def positional_encoding(self, position, d_model):\n    # 각도 배열 생성\n    angle_rads = self.get_angles(\n        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],\n        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],\n        d_model=d_model)\n\n    # 배열의 짝수 인덱스에는 sin 함수 적용\n    sines = tf.math.sin(angle_rads[:, 0::2])\n    # 배열의 홀수 인덱스에는 cosine 함수 적용\n    cosines = tf.math.cos(angle_rads[:, 1::2])\n\n    # sin과 cosine이 교차되도록 재배열\n    pos_encoding = tf.stack([sines, cosines], axis=0)\n    pos_encoding = tf.transpose(pos_encoding,[1, 2, 0]) \n    pos_encoding = tf.reshape(pos_encoding, [p

In [20]:
# 스케일드 닷 프로덕트 어텐션 함수


def scaled_dot_product_attention(query, key, value, mask):
  # 어텐션 가중치는 Q와 K의 닷 프로덕트
    matmul_qk = tf.matmul(query, key, transpose_b=True)

  # 가중치를 정규화
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)

  # 패딩에 마스크 추가
    if mask is not None:
        logits += (mask * -1e9)

  # softmax적용
    attention_weights = tf.nn.softmax(logits, axis=-1)

  # 최종 어텐션은 가중치와 V의 닷 프로덕트
    output = tf.matmul(attention_weights, value)
    return output

In [21]:
## gpt에서는 sin,cos로 위치를 구하지 않고 layer We를 통해 구함
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, max_position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_embedding = self.add_weight("pos_embedding", shape=(max_position, d_model))

    def call(self, inputs):
        batch_size, seq_length, _ = tf.shape(inputs)
        return inputs + self.pos_embedding[:seq_length, :]  # 시퀀스 길이에 맞는 위치 임베딩을 추가


In [22]:
class FeedForwardNetwork(Layer):
    def __init__(self, d_model, d_ff, rate):
        super(FeedForwardNetwork, self).__init__()
        self.dense1 = Dense(d_ff, activation='relu')
        self.dense2 = Dense(d_model)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)
        self.layer_norm = LayerNormalization(epsilon=1e-6)

    def call(self, inputs, training):
        x = self.dense1(inputs)
        x = self.dropout1(x, training=training)
        x = self.dense2(x)
        x = self.dropout2(x, training=training)

        return self.layer_norm(inputs + x)  # Residual connection and layer norm


In [23]:
# TransformerBlock 수정
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.attention = MultiHeadAttention(d_model, num_heads)
        self.ffn = FeedForwardNetwork(d_model, dff, rate)
        self.layer_norm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layer_norm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training, mask=None):
        # Ensure inputs are processed as needed
        attn_output = self.attention({
            'query': inputs,
            'key': inputs,
            'value': inputs,
            'mask': mask
        })  # Pass the same inputs for Q, K, V
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layer_norm1(inputs + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layer_norm2(out1 + ffn_output)


In [24]:
# MultiHeadAttention 클래스 수정
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)

        self.dense = tf.keras.layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(
            inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query = self.query_dense(inputs['query'])
        key = self.key_dense(inputs['key'])
        value = self.value_dense(inputs['value'])
        mask = inputs.get('mask')

        batch_size = tf.shape(query)[0]

        # 병렬 연산을 위한 머리를 여러 개 만듭니다
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # 스케일드 닷 프로덕트 어텐션 함수
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        # 어텐션 연산 후에 각 결과를 다시 연결(concatenate)합니다
        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))

        # 최종 결과에도 Dense를 한 번 더 적용합니다
        outputs = self.dense(concat_attention)

        return outputs


In [25]:
def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
    return mask[:, tf.newaxis, tf.newaxis, :]

In [26]:
def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)

## 모델 구성하기

In [27]:
# 인코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 두 개의 서브 레이어가 존재합니다.
## gpt -> encoder x
"""

def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

  # 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })

  # 어텐션의 결과는 Dropout과 Layer Normalization이라는 훈련을 돕는 테크닉을 수행
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  # 두 번째 서브 레이어 : 2개의 완전연결층
  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 완전연결층의 결과는 Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)
"""

'\n\ndef encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):\n  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")\n\n  # 패딩 마스크 사용\n  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")\n\n  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)\n  attention = MultiHeadAttention(\n      d_model, num_heads, name="attention")({\n          \'query\': inputs,\n          \'key\': inputs,\n          \'value\': inputs,\n          \'mask\': padding_mask\n      })\n\n  # 어텐션의 결과는 Dropout과 Layer Normalization이라는 훈련을 돕는 테크닉을 수행\n  attention = tf.keras.layers.Dropout(rate=dropout)(attention)\n  attention = tf.keras.layers.LayerNormalization(\n      epsilon=1e-6)(inputs + attention)\n\n  # 두 번째 서브 레이어 : 2개의 완전연결층\n  outputs = tf.keras.layers.Dense(units=units, activation=\'relu\')(attention)\n  outputs = tf.keras.layers.Dense(units=d_model)(outputs)\n\n  # 완전연결층의 결과는 Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행\n  outputs = tf.keras.layers.Dropout(rate=dropout)(o

In [28]:
## gpt model에서는 encoder 부분은 필요 없음 
"""
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 임베딩 레이어
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  # 포지셔널 인코딩
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # num_layers만큼 쌓아올린 인코더의 층.
  for i in range(num_layers):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)
"""

'\ndef encoder(vocab_size,\n            num_layers,\n            units,\n            d_model,\n            num_heads,\n            dropout,\n            name="encoder"):\n  inputs = tf.keras.Input(shape=(None,), name="inputs")\n\n  # 패딩 마스크 사용\n  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")\n\n  # 임베딩 레이어\n  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)\n  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))\n\n  # 포지셔널 인코딩\n  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)\n\n  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)\n\n  # num_layers만큼 쌓아올린 인코더의 층.\n  for i in range(num_layers):\n    outputs = encoder_layer(\n        units=units,\n        d_model=d_model,\n        num_heads=num_heads,\n        dropout=dropout,\n        name="encoder_layer_{}".format(i),\n    )([outputs, padding_mask])\n\n  return tf.keras.Model(\n      inputs=[inputs, padding_mask], outputs=outputs, name=name)\n'

In [29]:
# 디코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 세 개의 서브 레이어가 존재합니다.
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
    look_ahead_mask = tf.keras.Input(
        shape=(1, None, None), name="look_ahead_mask")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
    attention1 = MultiHeadAttention(
        d_model, num_heads, name="attention_1")(inputs={
            'query': inputs,
            'key': inputs,
            'value': inputs,
            'mask': look_ahead_mask
         })

  # 멀티 헤드 어텐션의 결과는 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
    attention1 = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(attention1 + inputs)

    
  # 두 번째 서브 레이어 : 마스크드 멀티 헤드 어텐션 수행 (인코더-디코더 어텐션)
  ## encoder가 없어짐으로 두 번째 sub layer도 필요 없어짐
    

    return tf.keras.Model(
        inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
        outputs=outputs,
        name=name)

In [30]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = tf.keras.Input(
        shape=(1, None, None), name='look_ahead_mask')

  # 패딩 마스크
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
  
  # 임베딩 레이어
    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  # 포지셔널 인코딩
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  # Dropout이라는 훈련을 돕는 테크닉을 수행
    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name='decoder_layer_{}'.format(i),
        )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return tf.keras.Model(
        inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
        outputs=outputs,
        name=name)

## 교사 강요(Teacher Forcing) 사용하기

In [31]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# 디코더는 이전의 target을 다음의 input으로 사용합니다.
# 이에 따라 outputs에서는 START_TOKEN을 제거하겠습니다.
"""
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1]
    },
    {
        'outputs': answers[:, 1:]
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
"""
# Tokenizer 설정
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(data, target_vocab_size=2**15)

# 데이터 인코딩
inputs = [tokenizer.encode(f"{q}${a}") for q, a in zip(dialogues, answers)]
outputs = [tokenizer.encode(a) for a in answers]  # 모든 답변을 포함

# 최대 시퀀스 길이 정의
max_length = 22

# 패딩 추가
inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs, maxlen=max_length, padding='post')
outputs = tf.keras.preprocessing.sequence.pad_sequences(outputs, maxlen=max_length, padding='post')  # 마지막 요소 제거 필요 없음

# 데이터셋 생성
dataset = tf.data.Dataset.from_tensor_slices((inputs, outputs))

# 데이터셋 캐시, 셔플, 배치 및 프리패치 설정
BUFFER_SIZE = 10000
BATCH_SIZE = 64
dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)






##  모델 정의 및 평가하기

## GPT Model 정의하기

In [32]:
class GPTModel(tf.keras.Model):
    def __init__(self, vocab_size, d_model, num_layers, num_heads, dff, max_position_encoding, rate=0.1):
        super(GPTModel, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, d_model)
        self.transformer_blocks = [TransformerBlock(d_model, num_heads, dff, rate) for _ in range(num_layers)]
        self.final_layer = tf.keras.layers.Dense(vocab_size)  # 최종 출력 형태 (None, 22, vocab_size)

    def call(self, inputs, training, mask=None):
        # 임베딩을 적용
        x = self.embedding(inputs)  # (batch_size, seq_length, d_model)
        
        for transformer in self.transformer_blocks:
            x = transformer(x, training, mask)  # 각 TransformerBlock의 출력
          
        return self.final_layer(x)  # 최종 레이어에서 vocab_size 차원으로 출력


In [33]:
# Define hyperparameters
vocab_size = tokenizer.vocab_size + 2
d_model = 128
num_layers = 4
num_heads = 8
dff = 512
max_position_encoding = 10
rate = 0.1

# Create the model
model = GPTModel(vocab_size, d_model, num_layers, num_heads, dff, max_position_encoding, rate)

# Define input shape for the model
input_shape = (None, 22)

# Build the model
model.build(input_shape)

# Print model summary
model.summary()


##??? output shape -> multiple ??  이게 맞나..

Model: "gpt_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  33280     
_________________________________________________________________
transformer_block (Transform multiple                  198528    
_________________________________________________________________
transformer_block_1 (Transfo multiple                  198528    
_________________________________________________________________
transformer_block_2 (Transfo multiple                  198528    
_________________________________________________________________
transformer_block_3 (Transfo multiple                  198528    
_________________________________________________________________
dense_24 (Dense)             multiple                  33540     
Total params: 860,932
Trainable params: 860,932
Non-trainable params: 0
___________________________________________________

In [34]:
learning_rate = 0.01  # 학습률 설정
sgd_optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
model.compile(optimizer= sgd_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [35]:
EPOCHS = 10
model.fit(dataset, epochs=EPOCHS, verbose=1)

Epoch 1/10
185/185 [==============================] - 77s 385ms/step - loss: 5.6078 - accuracy: 0.0040
Epoch 2/10
185/185 [==============================] - 71s 385ms/step - loss: 5.5508 - accuracy: 0.0032
Epoch 3/10
185/185 [==============================] - 71s 384ms/step - loss: 4.9927 - accuracy: 0.0031
Epoch 4/10
185/185 [==============================] - 71s 386ms/step - loss: 4.9874 - accuracy: 0.0031
Epoch 5/10
185/185 [==============================] - 72s 387ms/step - loss: 4.9942 - accuracy: 0.0031
Epoch 6/10
185/185 [==============================] - 71s 383ms/step - loss: 5.0021 - accuracy: 0.0031
Epoch 7/10
185/185 [==============================] - 72s 387ms/step - loss: 5.0069 - accuracy: 0.0031
Epoch 8/10
185/185 [==============================] - 71s 384ms/step - loss: 5.0097 - accuracy: 0.0031
Epoch 9/10
185/185 [==============================] - 71s 384ms/step - loss: 5.0154 - accuracy: 0.0031
Epoch 10/10
185/185 [==============================] - 72s 389ms/step - l

## Chatbot Test

In [39]:
def decoder_inference(sentence):
    sentence = preprocess_sentence(sentence)

    # 입력된 문장을 정수 인코딩 후, 시작 토큰과 종료 토큰을 앞뒤로 추가.
    sentence = START_TOKEN + tokenizer.encode(sentence) + END_TOKEN
    sentence = tf.convert_to_tensor([sentence], dtype=tf.int32)  # 리스트 -> 텐서 변환

    # 디코더의 현재까지의 예측한 출력 시퀀스가 지속적으로 저장되는 변수.
    output_sequence = tf.convert_to_tensor([START_TOKEN], dtype=tf.int32)  # 리스트 -> 텐서 변환

    # 디코더의 인퍼런스 단계
    for i in range(MAX_LENGTH):
        # 모델에 전달할 때 inputs 리스트를 텐서로 변환
        predictions = model(inputs=[sentence, output_sequence], training=False)
        predictions = predictions[:, -1:, :]  # 마지막 토큰 예측

        # 현재 예측한 단어의 정수
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        # 만약 현재 예측한 단어가 종료 토큰이라면 for문을 종료
        if tf.equal(predicted_id, END_TOKEN[0]):
            break

        # 예측한 단어들을 지속적으로 output_sequence에 추가
        output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)

    return tf.squeeze(output_sequence, axis=0)  # 불필요한 차원 제거

In [45]:
def decoder_inference(sentence):
    sentence = preprocess_sentence(sentence)

    # 문장을 정수 인코딩 후, 시작 토큰과 종료 토큰을 추가
    sentence = START_TOKEN + tokenizer.encode(sentence) + END_TOKEN
    sentence = tf.convert_to_tensor([sentence], dtype=tf.int32)

    # 디코더의 예측된 출력 시퀀스를 저장할 변수 (시작 토큰으로 초기화)
    output_sequence = tf.convert_to_tensor([START_TOKEN], dtype=tf.int32)

    for i in range(MAX_LENGTH):
        # 🔹 sentence와 output_sequence를 하나의 텐서로 병합
        model_input = tf.concat([sentence, output_sequence], axis=-1)

        # 모델 예측
        predictions = model(inputs=model_input, training=False)
        predictions = predictions[:, -1:, :]  # 마지막 토큰 예측

        # 현재 예측한 단어의 정수 변환
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        # 종료 토큰이면 중단
        if tf.equal(predicted_id, END_TOKEN[0]):
            break

        # output_sequence에 새로운 예측 추가
        output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)

    return tf.squeeze(output_sequence, axis=0)  # 불필요한 차원 제거


In [46]:
def sentence_generation(sentence):
    # 입력 문장에 대해서 디코더를 동작 시켜 예측된 정수 시퀀스를 리턴받습니다.
    prediction = decoder_inference(sentence)

    # 정수 시퀀스를 다시 텍스트 시퀀스로 변환합니다.
    predicted_sentence = tokenizer.decode(
        [i for i in prediction if i < tokenizer.vocab_size])

    print('입력 : {}'.format(sentence))
    print('출력 : {}'.format(predicted_sentence))

    return predicted_sentence

In [47]:
## drop out 0.5, max_length = 10
sentence_generation('안녕')

InvalidArgumentError: indices[0,0] = 8164 is not in [0, 260) [Op:ResourceGather]

In [48]:
sentence_generation('오늘 날씨 어때')

InvalidArgumentError: indices[0,0] = 8164 is not in [0, 260) [Op:ResourceGather]

In [49]:
sentence_generation('오늘 뭐 먹어?')

InvalidArgumentError: indices[0,0] = 8164 is not in [0, 260) [Op:ResourceGather]